<a href="https://colab.research.google.com/github/daniilprigozhin/ProteinFamily/blob/main/Protein_Family_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Protein family analysis in Google Colab

This project will show you how to construct protein family phylogeny using NLR family as an example. The principal advantage of colab is ability to follow along and modify code as needed. We have implemented similar pipelines using Snakemake for execution on local machines.


##Colab basics
To run a section of code
* Hit play button OR
* Hit Cmd/Cntrl + Enter

To edit a section of code/text
* Double click the code/text window

##Step 0: Install the software 
We'll be using

HMMER with easel tools: 
http://hmmer.org

Prank: 
http://wasabiapp.org/software/prank/

Belvu (not in Colab): 
https://www.sanger.ac.uk/resources/software/seqtools/

RAxML: 
https://cme.h-its.org/exelixis/web/software/raxml/


In [1]:
##This block takes ~4 minutes
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c bioconda hmmer 
!conda install -c bioconda easel
#!conda install -c bioconda::snakemake
!conda install -c bioconda raxml-ng
#!conda update raxml-ng

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:39
🔁 Restarting kernel...
Solving environment: \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - hmmer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_2         3.1 MB  conda-forge
    hmmer-3.3.2                |       h1b792b2_1         9.6 MB  bioconda
    ------------------------------------------------------------
                                           Total:        13.0 MB

The following NEW packages will be INSTAL


##Step 1: Load Proteome and Domain Model 

###Load proteome of your species of interest 
Here we will use protein models from Van de Weyer et al, 2019 Cell.


**Phytozome** is a good source for plant protein models. 
Example: rice
https://phytozome.jgi.doe.gov/pz/portal.html#!info?alias=Org_Osativa
Click to Bulk data download, login in and you will find yourself at FTP
You can find proteome under /annotation/Osativa_323_v7.0.protein.fa.gz 

**Uniprot** has convenient one protein per gene proteomes available for download for "all things bright and beautiful".

To load proteomes into colab one can either place them in **GitHub** and use git clone, or use the Files -> Upload to Session Storage (click **folder** icon on the left and then click **paper with up arrow** icon).

In [77]:
%rm -rf ProteinFamily/
!git clone https://github.com/daniilprigozhin/ProteinFamily.git
!ls

Cloning into 'ProteinFamily'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 164 (delta 93), reused 130 (delta 70), pack-reused 0
Receiving objects: 100% (164/164), 4.69 MiB | 24.75 MiB/s, done.
Resolving deltas: 100% (93/93), done.
108.aa.fa		       pbNB-ARC.raxml.bestTree
108.aa.fa.ssi		       pbNB-ARC.raxml.bestTreeCollapsed
108.iTOL.domains.txt	       pbNB-ARC.raxml.bootstraps
108.pbNB-ARC.fulllength.fa     pbNB-ARC.raxml.log
108.pbNB-ARC.list	       pbNB-ARC.raxml.mlTrees
108.pbNB-ARC.Pfam.reduced.tbl  pbNB-ARC.raxml.rba
108.pbNB-ARC.Pfam.tbl	       pbNB-ARC.raxml.reduced.phy
108.pbNB-ARC.Pfam.ws.tbl       pbNB-ARC.raxml.startTree
All_Domains_with_colors.tsv    pbNB-ARC.raxml.support
Colab.RData		       Pfam-A.hmm
condacolab_install.log	       Pfam-A.plus.hmm
drive			       Pfam-A.plus.hmm.h3f
NB-ARC.hmm		       Pfam-A.plus.hmm.h3i
pbNB-ARC.hmm		       Pfam-A.plus.hmm.h3m
pbNB


###Load a statistical model for your domain of interest

Since all NLRs have a conserved NB-ARC domain, you can extract proteins containing this domain’s HMM from plant proteome and align them using HMM as a template. Go to http://pfam.xfam.org/family/NB-ARC 
and download http://pfam.xfam.org/family/PF00931/hmm

To load this hmm file into colab:

In [3]:
!wget -O NB-ARC.hmm http://pfam.xfam.org/family/PF00931/hmm 

--2021-10-02 00:45:07--  http://pfam.xfam.org/family/PF00931/hmm
Resolving pfam.xfam.org (pfam.xfam.org)... 193.62.193.83
Connecting to pfam.xfam.org (pfam.xfam.org)|193.62.193.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118379 (116K) [text/plain]
Saving to: ‘NB-ARC.hmm’

NB-ARC.hmm          100%[===================>] 115.60K   315KB/s    in 0.4s    

2021-10-02 00:45:08 (315 KB/s) - ‘NB-ARC.hmm’ saved [118379/118379]




####What is HMM?
If you are curious - look inside the .hmm file to see how the domain is described as a statistical model of aa probabilities at each position of the domain.
####What if you need a new HMM?
What if your protein of interest does not have pre-built HMM describing it? You can build HMM yourself using `hmmbuild` function in HMMER.

What if HMM at Pfam does not adequately describe your protein family of interest. For example, the NB-ARC model above has been built from diverse organisms including not only plants but also animals, bacteria and archaea. Therefore the resulting HMM is a best fit to describe full diversity of this protein family.

Most of plant NLRs have a conserved NB-ARC domain that is larger than what Pfam HMM describes. NB-ARC of plant NLRs include additional motifs such as ARC2 and MHD. Therefore, we built plant specific HMM that you can download here: 

Bailey et al, Genome Biology 2018, Additional file 16:

In [ ]:
!wget -O pbNB-ARC.hmm https://static-content.springer.com/esm/art%3A10.1186%2Fs13059-018-1392-6/MediaObjects/13059_2018_1392_MOESM16_ESM.hmm

You can do the alignment steps below with both Pfam HMM and our HMM and compare the results.

As a bonus, you can download any of the curated functionally annotated NLRs from http://prgdb.crg.eu/wiki/Category:Reference_R-Genes,_manually_curated and include them in alignment and phylogeny. Place them in project folder as well.


##Step 2: Align proteins to Model

We will use HMMER to align proteins to model:  

    hmmsearch -E 1e-5 -A <domain.hmmalign.sto> <domain>.hmm <proteome>.faa
                                                  

In [ ]:
!hmmsearch -E 1e-5 -A pbNB-ARC.hmmalign.sto --domtblout pbNB-ARC.hmmalign.tbl ProteinFamily/HMM_models/pbNB-ARC.hmm ProteinFamily/Proteomes/108.aa.fa
!cat pbNB-ARC.hmmalign.sto

Hmmalign produces an alignment in Stockholm format, however for visualisation and tree building we need the alignment in fasta format. 

`esl-alimask --rf-is-mask` removes columns that do not match model

`esl-alimanip --lmin` removes rows that are shorter than a user-defined threshold (in this case 237aa = 70% of the model length for our custom NB-ARC HMM)

`esl-reformat` reformats to fasta format. The same tool can also trim the alignment to remove insertions and short sequences.

`-` in easel signals that input to the command will come in from the pipe

Finally `cut` and `tr` remove extra fields in the protein names

In [ ]:
!esl-alimask --rf-is-mask pbNB-ARC.hmmalign.sto | esl-alimanip --lmin 100 -|esl-reformat afa - |cut -d ' ' -f 1 |tr -d ' ' > pbNB-ARC.hmmalign.afa
!cat pbNB-ARC.hmmalign.afa

##Step 3: Phylogeny with RAXML

We are now ready to build a tree of the protein domains to visualise how they may be related evolutionarily. For this we are going to use the RAXML to build a bootstrapped maximum likelihood tree. This will take >2 hours but will actually work! Skip ahead to load precomputed results.

In [4]:
!raxml-ng --all --bs-trees 100 --model JTT --prefix pbNB-ARC --msa pbNB-ARC.hmmalign.afa 


RAxML-NG v. 1.0.3 released on 21.07.2021 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Intel(R) Xeon(R) CPU @ 2.20GHz, 1 cores, 12 GB RAM

RAxML-NG was called at 02-Oct-2021 01:50:25 as follows:

raxml-ng --all --bs-trees 100 --model JTT --prefix pbNB-ARC --msa pbNB-ARC.hmmalign.afa

Analysis options:
  run mode: ML tree search + bootstrapping (Felsenstein Bootstrap)
  start tree(s): random (10) + parsimony (10)
  bootstrap replicates: 100
  random seed: 1633139425
  tip-inner: OFF
  pattern compression: ON
  per-rate scalers: OFF
  site repeats: ON
  branch lengths: proportional (ML estimate, algorithm: NR-FAST)
  SIMD kernels: AVX2
  parallelization: coarse-grained (auto), NONE/sequential

[00:00:00] Reading a

In [5]:
## If you need to get the precomputed tree, unquote and run this line:
#!cp ProteinFamily/Colab_Results/pbNB-ARC.raxml.* .
!ls

condacolab_install.log		  pbNB-ARC.raxml.bootstraps
NB-ARC.hmm			  pbNB-ARC.raxml.log
pbNB-ARC.hmm			  pbNB-ARC.raxml.mlTrees
pbNB-ARC.hmmalign.afa		  pbNB-ARC.raxml.rba
pbNB-ARC.hmmalign.sto		  pbNB-ARC.raxml.reduced.phy
pbNB-ARC.hmmalign.tbl		  pbNB-ARC.raxml.startTree
pbNB-ARC.raxml.bestModel	  pbNB-ARC.raxml.support
pbNB-ARC.raxml.bestTree		  ProteinFamily
pbNB-ARC.raxml.bestTreeCollapsed  sample_data


##Step 4: Saving Results
You can connect to ***your own*** Google Drive and save any results you'd like to keep. 

In [11]:
!ls
from google.colab import drive
drive.mount('/content/drive')
!cp pbNB-ARC* /content/drive/MyDrive/Colab_Results/

condacolab_install.log		  pbNB-ARC.raxml.bootstraps
drive				  pbNB-ARC.raxml.log
NB-ARC.hmm			  pbNB-ARC.raxml.mlTrees
pbNB-ARC.hmm			  pbNB-ARC.raxml.rba
pbNB-ARC.hmmalign.afa		  pbNB-ARC.raxml.reduced.phy
pbNB-ARC.hmmalign.sto		  pbNB-ARC.raxml.startTree
pbNB-ARC.hmmalign.tbl		  pbNB-ARC.raxml.support
pbNB-ARC.raxml.bestModel	  ProteinFamily
pbNB-ARC.raxml.bestTree		  sample_data
pbNB-ARC.raxml.bestTreeCollapsed
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!ls /content/drive/MyDrive/Colab_Results/

pbNB-ARC.hmm			  pbNB-ARC.raxml.bootstraps
pbNB-ARC.hmmalign.afa		  pbNB-ARC.raxml.log
pbNB-ARC.hmmalign.sto		  pbNB-ARC.raxml.mlTrees
pbNB-ARC.hmmalign.tbl		  pbNB-ARC.raxml.rba
pbNB-ARC.raxml.bestModel	  pbNB-ARC.raxml.reduced.phy
pbNB-ARC.raxml.bestTree		  pbNB-ARC.raxml.startTree
pbNB-ARC.raxml.bestTreeCollapsed  pbNB-ARC.raxml.support


##Step 5: The fun part - annotating your tree
In this section we will annotate Pfam domains in our proteins of interest and will call a couple of R scripts to produce an annotation file for iTOL tree viewer. The motivation is to check what other domains are present in our proteins of interest (that all share the NB-ARC domain). 

Step 5.1: Load Pfam (supplement with your favorite domains using `cat`). Prepare the local pfam for running.

In [13]:
!wget ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz
!gunzip Pfam-A.hmm.gz
!cat Pfam-A.hmm ProteinFamily/HMM_models/* > Pfam-A.plus.hmm
!hmmpress Pfam-A.plus.hmm

--2021-10-02 05:42:35--  ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz
           => ‘Pfam-A.hmm.gz’
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.197.74
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/Pfam/current_release ... done.
==> SIZE Pfam-A.hmm.gz ... 286543529
==> PASV ... done.    ==> RETR Pfam-A.hmm.gz ... done.
Length: 286543529 (273M) (unauthoritative)

Pfam-A.hmm.gz       100%[===================>] 273.27M  32.4MB/s    in 9.1s    

2021-10-02 05:42:46 (30.2 MB/s) - ‘Pfam-A.hmm.gz’ saved [286543529]

Working...    done.
Pressed and indexed 19181 HMMs (19181 names and 19179 accessions).
Models pressed into binary file:   Pfam-A.plus.hmm.h3m
SSI index for binary model file:   Pfam-A.plus.hmm.h3i
Profiles (MSV part) pressed into:  Pfam-A.plus.hmm.h3f
Profiles (remainder) pressed into: Pfam-A.plus

Step 5.2: use hmm-search to find Pfam domains in your protein collection

In [81]:
## There is a perl script in here that's been perfect since the dawn of time
## I tried to do this simple task with easel tools and failed miserably
!grep '>' pbNB-ARC.hmmalign.afa|cut -f 1 -d '/'| tr -d '>' |sort |uniq >108.pbNB-ARC.list
!wc 108.pbNB-ARC.list
!ProteinFamily/scripts/K-get_fasta_from_ids.pl -f ProteinFamily/Proteomes/108.aa.fa -i 108.pbNB-ARC.list > 108.pbNB-ARC.fulllength.fa
!grep '>' 108.pbNB-ARC.fulllength.fa|wc 

 168  168 2038 108.pbNB-ARC.list
    168     168    2374


In [80]:
!hmmsearch --domtblout 108.pbNB-ARC.Pfam.tbl Pfam-A.plus.hmm 108.pbNB-ARC.fulllength.fa

Streaming output truncated to the last 5000 lines.
Passed Vit filter:                         0  (0); expected 0.2 (0.001)
Passed Fwd filter:                         0  (0); expected 0.0 (1e-05)
Initial search space (Z):                168  [actual number of targets]
Domain search space  (domZ):               0  [number of targets reported over threshold]
# CPU time: 0.00u 0.00s 00:00:00.00 Elapsed: 00:00:00.00
# Mc/sec: 4055.91
//
Query:       Zw10  [M=527]
Accession:   PF06248.15
Description: Centromere/kinetochore Zw10
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence Description
    ------- ------ -----    ------- ------ -----   ---- --  -------- -----------

   [No hits detected that satisfy reporting thresholds]


Domain annotation for each sequence (and alignments):

   [No targets detected that satisfy reporting thresholds]


Internal pipelin

In [82]:
!cat 108.pbNB-ARC.Pfam.tbl|grep -v '#'|wc

   5436  125028  994788


In [83]:
!cp pbNB-ARC* /content/drive/MyDrive/Colab_Results/
!cp 108* /content/drive/MyDrive/Colab_Results/

In [46]:
%rm -rf ProteinFamily/
!git clone https://github.com/daniilprigozhin/ProteinFamily.git

Cloning into 'ProteinFamily'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 160 (delta 92), reused 127 (delta 70), pack-reused 0
Receiving objects: 100% (160/160), 4.69 MiB | 24.12 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [84]:
## The first tr command collapses spaces in hmmer output. Usually, in R read_delim() does this, but not in Colab for some reason.
!tr -s ' ' <108.pbNB-ARC.Pfam.tbl > 108.pbNB-ARC.Pfam.ws.tbl
!Rscript ProteinFamily/scripts/reduce_pfam.R -i 108.pbNB-ARC.Pfam.ws.tbl -o 108.pbNB-ARC.Pfam.reduced.tbl -e 1e-3 -f 0.3 -a 10


Loading required package: optparse
Loading required package: tidyverse
Failed to create bus connection: No such file or directory
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
In system("timedatectl", intern = TRUE) :
  running command 'timedatectl' had status 1
Loaded packages
Read table:
# A tibble: 5,436 × 23
   target_name t_accession  tlen query_name q_accession  qlen fullseq_Evalue
   <chr>       <chr>       <dbl> <chr>      <chr>       <dbl>          <dbl>
 1 108_T241-R1 -            1608 AAA        PF00004.31    132  0.00000000005
 2 108_T241-R1 -            1608 AAA        PF00004.31    132  0.00000000005
 3 108_T105-R1 -        

In [87]:
!Rscript ProteinFamily/scripts/DomainDiagrams_sm.R -o 108.iTOL.domains.txt -i 108.pbNB-ARC.Pfam.reduced.tbl -f 108.pbNB-ARC.fulllength.fa -a pbNB-ARC.hmmalign.afa


'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.13 (BiocManager 1.30.16), R 4.1.1 (2021-08-10)
Old packages: 'cpp11', 'data.table', 'desc', 'digest', 'hms', 'knitr',
  'lifecycle', 'mime', 'openssl', 'pillar', 'rcmdcheck', 'readr', 'remotes',
  'tibble', 'tidyr', 'tinytex', 'nlme'
Warning message:
package(s) not installed when version(s) same as current; use `force = TRUE` to
  re-install: 'Biostrings' 
Loading required package: optparse
Loading required package: tidyverse
Failed to create bus connection: No such file or directory
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() mas

In [86]:
!grep '>' 108.pbNB-ARC.fulllength.fa|wc
!grep '>' pbNB-ARC.hmmalign.afa|wc

    168     168    2374
    174     174    3664
